In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
from sklearn.model_selection import TimeSeriesSplit
from fbprophet import Prophet

In [2]:
dfw = pd.read_csv("../write_data/Dallas-Fort Worth.csv")

In [3]:
zippies = dfw.Zip_code.unique()

In [32]:
df = pd.read_csv("../write_data/zillow_housing_zipcodes.csv")

In [33]:
df.head()

,Date_index,60657,75070,77494,60614,79936,77084,10467,60640,77449,...,3765,84781,12429,97028,12720,1338,3293,40404,81225,89155
0,1996-04-01,334200.0,235700.0,210400.0,498100.0,77300.0,95000.0,152900.0,216500.0,95400.0,...,80800.0,135900.0,78300.0,136200.0,62500.0,94600.0,92700.0,57100.0,191100.0,176400.0
1,1996-05-01,335400.0,236900.0,212200.0,500900.0,77300.0,95200.0,152700.0,216700.0,95600.0,...,80100.0,136300.0,78300.0,136600.0,62600.0,94300.0,92500.0,57300.0,192400.0,176300.0
2,1996-06-01,336500.0,236700.0,212200.0,503100.0,77300.0,95400.0,152600.0,216900.0,95800.0,...,79400.0,136600.0,78200.0,136800.0,62700.0,94000.0,92400.0,57500.0,193700.0,176100.0
3,1996-07-01,337600.0,235400.0,210700.0,504600.0,77300.0,95700.0,152400.0,217000.0,96100.0,...,78600.0,136900.0,78200.0,136800.0,62700.0,93700.0,92200.0,57700.0,195000.0,176000.0
4,1996-08-01,338500.0,233300.0,208300.0,505500.0,77400.0,95900.0,152300.0,217100.0,96400.0,...,77900.0,137100.0,78100.0,136700.0,62700.0,93400.0,92100.0,58000.0,196300.0,175900.0


In [34]:
df['Date_index'] = pd.to_datetime(df['Date_index'])
df = df.set_index('Date_index')
df.head()

,60657,75070,77494,60614,79936,77084,10467,60640,77449,94109,...,3765,84781,12429,97028,12720,1338,3293,40404,81225,89155
Date_index,,,,,,,,,,,,,,,,,,,,,
1996-04-01,334200.0,235700.0,210400.0,498100.0,77300.0,95000.0,152900.0,216500.0,95400.0,766000.0,...,80800.0,135900.0,78300.0,136200.0,62500.0,94600.0,92700.0,57100.0,191100.0,176400.0
1996-05-01,335400.0,236900.0,212200.0,500900.0,77300.0,95200.0,152700.0,216700.0,95600.0,771100.0,...,80100.0,136300.0,78300.0,136600.0,62600.0,94300.0,92500.0,57300.0,192400.0,176300.0
1996-06-01,336500.0,236700.0,212200.0,503100.0,77300.0,95400.0,152600.0,216900.0,95800.0,776500.0,...,79400.0,136600.0,78200.0,136800.0,62700.0,94000.0,92400.0,57500.0,193700.0,176100.0
1996-07-01,337600.0,235400.0,210700.0,504600.0,77300.0,95700.0,152400.0,217000.0,96100.0,781900.0,...,78600.0,136900.0,78200.0,136800.0,62700.0,93700.0,92200.0,57700.0,195000.0,176000.0
1996-08-01,338500.0,233300.0,208300.0,505500.0,77400.0,95900.0,152300.0,217100.0,96400.0,787300.0,...,77900.0,137100.0,78100.0,136700.0,62700.0,93400.0,92100.0,58000.0,196300.0,175900.0


In [38]:
df_annual = df.resample("YS").mean()

In [39]:
df_annual

,60657,75070,77494,60614,79936,77084,10467,60640,77449,94109,...,3765,84781,12429,97028,12720,1338,3293,40404,81225,89155
Date_index,,,,,,,,,,,,,,,,,,,,,
1996-01-01,3.384444e+05,230988.888889,206655.555556,5.034444e+05,77455.555556,95733.333333,152211.111111,217266.666667,96255.555556,7.883444e+05,...,77877.777778,137122.222222,78133.333333,136433.333333,62555.555556,93533.333333,91966.666667,58000.000000,196511.111111,176055.555556
1997-01-01,3.527000e+05,198875.000000,200100.000000,4.990833e+05,77933.333333,95616.666667,152308.333333,227275.000000,97008.333333,8.831250e+05,...,71641.666667,140983.333333,78350.000000,132125.000000,62183.333333,94183.333333,89258.333333,62416.666667,218016.666667,177183.333333
1998-01-01,3.967667e+05,205291.666667,240391.666667,5.616250e+05,79216.666667,97608.333333,156716.666667,260908.333333,101333.333333,1.070767e+06,...,69458.333333,145008.333333,79566.666667,119941.666667,62466.666667,102083.333333,87283.333333,70958.333333,263175.000000,180150.000000
1999-01-01,4.671083e+05,197375.000000,301125.000000,6.519167e+05,81075.000000,104441.666667,166766.666667,313700.000000,112208.333333,1.372342e+06,...,69375.000000,147516.666667,81683.333333,116175.000000,64375.000000,109908.333333,88666.666667,77475.000000,313416.666667,184808.333333
2000-01-01,5.658833e+05,222158.333333,319941.666667,7.513833e+05,82450.000000,105458.333333,183783.333333,379883.333333,124491.666667,1.756550e+06,...,69150.000000,150358.333333,88000.000000,119975.000000,67883.333333,116533.333333,93433.333333,79375.000000,352591.666667,193741.666667
2001-01-01,6.539833e+05,216750.000000,290216.666667,8.538167e+05,79608.333333,107283.333333,204675.000000,440116.666667,116650.000000,2.029275e+06,...,74325.000000,154366.666667,100433.333333,123291.666667,75266.666667,125083.333333,101291.666667,82833.333333,383575.000000,207125.000000
2002-01-01,7.117583e+05,194191.666667,238033.333333,9.321750e+05,79266.666667,112166.666667,230691.666667,489150.000000,115983.333333,2.135667e+06,...,83575.000000,156008.333333,119125.000000,130166.666667,87016.666667,136925.000000,113908.333333,85016.666667,406350.000000,222783.333333
2003-01-01,7.443750e+05,188608.333333,221733.333333,9.941833e+05,81983.333333,116408.333333,250266.666667,525333.333333,120125.000000,2.111233e+06,...,94291.666667,153108.333333,151000.000000,143941.666667,97341.666667,153308.333333,132841.666667,89125.000000,416425.000000,248766.666667
2004-01-01,7.840667e+05,193591.666667,232491.666667,1.039658e+06,88375.000000,122700.000000,286091.666667,559783.333333,127916.666667,2.368533e+06,...,99383.333333,153858.333333,178733.333333,190558.333333,117516.666667,177575.000000,170666.666667,96025.000000,462841.666667,348575.000000


In [64]:
metro_annual = df_annual.mean(axis=1)

In [73]:
metro_annual.pct_change().diff()

Date_index
1996-01-01         NaN
1997-01-01         NaN
1998-01-01    0.027987
1999-01-01    0.016517
2000-01-01    0.018673
2001-01-01   -0.005099
2002-01-01   -0.004632
2003-01-01    0.010561
2004-01-01    0.030880
2005-01-01    0.007745
2006-01-01   -0.066293
2007-01-01   -0.077044
2008-01-01   -0.061551
2009-01-01   -0.008843
2010-01-01    0.039365
2011-01-01   -0.007076
2012-01-01    0.046744
2013-01-01    0.064928
2014-01-01   -0.002021
2015-01-01   -0.003792
2016-01-01    0.002889
2017-01-01    0.006850
2018-01-01   -0.019274
Freq: AS-JAN, dtype: float64

In [66]:
df_annual[str(zippies[0])].pct_change().diff().mean()

0.008456034604763

In [67]:
dfw_apct = [(str(zippy), df_annual[str(zippy)].pct_change().diff().mean()) for zippy in zippies]
dfw_apct

[('75070', 0.008456034604763),
 ('75052', -0.00017205059970875914),
 ('75034', 0.002640109988031029),
 ('75287', 0.0002379257868178),
 ('75243', 0.003334889602910184),
 ('75067', 0.002601947701495334),
 ('76028', 0.0023174589256598314),
 ('76063', 0.001375765408887413),
 ('75228', 0.003674432495635525),
 ('76244', -0.002297386895540086),
 ('75056', 0.0024543038051786105),
 ('75217', 0.004386332898687136),
 ('75206', 0.0010014274593639137),
 ('75035', 0.010495504761636895),
 ('75150', 0.0026796896265800243),
 ('76116', 0.0035608291283206117),
 ('75043', 0.002208280208339461),
 ('75002', 0.0009499987731741274),
 ('75211', 0.0027128984973655407),
 ('75204', 0.00109628725245198),
 ('76179', -0.0007827640577909275),
 ('76137', 0.00178933753748375),
 ('76051', 0.0019159490216374446),
 ('75098', 0.0018535553170588223),
 ('75115', 0.0026304038368676974),
 ('75093', 0.00043320578530476564),
 ('75040', 0.004043028149457915),
 ('75007', 0.0017943365004349232),
 ('75149', 0.0011696443715723405),
 

In [68]:
dfw_apct_df = pd.DataFrame(dfw_apct, columns=['zip_code', 'annual_pct_change'])

In [69]:
dfw_five = dfw_apct_df.nlargest(5, "annual_pct_change")

In [70]:
dfw_five

,zip_code,annual_pct_change
193,75210,0.054889
135,75078,0.028746
13,75035,0.010496
126,76114,0.008876
0,75070,0.008456


In [71]:
dfw[dfw.Zip_code == 75210]

,Unnamed: 0,RegionName,City,State,Metro,CountyName,time,value,ds,yhat_lower,yhat,yhat_upper,Zip_code,log_value,log_yhat
193,193,75210,Dallas,TX,Dallas-Fort Worth,Dallas,1996-04-01,NaN,1996-04-01,192659.719772,194333.370463,196072.678922,75210,NaN,12.177330
410,410,75210,Dallas,TX,Dallas-Fort Worth,Dallas,1996-05-01,NaN,1996-05-01,193090.214618,194771.156903,196446.396611,75210,NaN,12.179581
627,627,75210,Dallas,TX,Dallas-Fort Worth,Dallas,1996-06-01,NaN,1996-06-01,192716.848644,194419.794136,196086.353922,75210,NaN,12.177775
844,844,75210,Dallas,TX,Dallas-Fort Worth,Dallas,1996-07-01,NaN,1996-07-01,190891.919140,192481.847523,194221.484411,75210,NaN,12.167757
1061,1061,75210,Dallas,TX,Dallas-Fort Worth,Dallas,1996-08-01,NaN,1996-08-01,188861.518094,190571.139677,192313.696765,75210,NaN,12.157781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56613,56613,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2017-12-01,70900.0,2017-12-01,70002.457741,71651.890044,73419.572795,75210,11.169026,11.179575
56830,56830,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2018-01-01,72900.0,2018-01-01,70559.525308,72200.421100,73889.570701,75210,11.196844,11.187201
57047,57047,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2018-02-01,75100.0,2018-02-01,71883.546564,73753.332274,75405.946595,75210,11.226576,11.208481
57264,57264,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2018-03-01,76900.0,2018-03-01,73389.925266,75142.549812,76825.569774,75210,11.250261,11.227142


In [49]:
dfw[dfw.Zip_code == 75210]['value'].isnull().sum()

213

In [ ]:
df[(df[Gender]=='Male') & (df[Year]==2014)]

In [61]:
dfw[(dfw['RegionName'] == 75210) & (dfw['time'] >= '2013-06-01')]

,Unnamed: 0,RegionName,City,State,Metro,CountyName,time,value,ds,yhat_lower,yhat,yhat_upper,Zip_code,log_value,log_yhat
44895,44895,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2013-06-01,NaN,2013-06-01,61988.314736,63755.939800,65510.962376,75210,NaN,11.062818
45112,45112,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2013-07-01,NaN,2013-07-01,60574.364627,62453.474181,64151.376449,75210,NaN,11.042177
45329,45329,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2013-08-01,NaN,2013-08-01,59405.804367,61123.773535,62907.114140,75210,NaN,11.020656
45546,45546,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2013-09-01,NaN,2013-09-01,58574.878551,60260.740035,62013.754998,75210,NaN,11.006436
45763,45763,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2013-10-01,NaN,2013-10-01,58342.794163,60082.074021,61731.668643,75210,NaN,11.003467
45980,45980,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2013-11-01,NaN,2013-11-01,58298.938071,60114.004392,61748.969002,75210,NaN,11.003998
46197,46197,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2013-12-01,NaN,2013-12-01,57446.587507,59257.875970,60984.901646,75210,NaN,10.989654
46414,46414,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2014-01-01,58600.0,2014-01-01,56383.603206,58088.651406,59818.266094,75210,10.978490,10.969726
46631,46631,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2014-02-01,58600.0,2014-02-01,56239.695783,57923.806959,59718.536739,75210,10.978490,10.966884
46848,46848,75210,Dallas,TX,Dallas-Fort Worth,Dallas,2014-03-01,57500.0,2014-03-01,55983.235718,57761.503291,59535.497228,75210,10.959540,10.964078
